In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 42.2 MB/s 
     |████████████████████████████████| 77 kB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 15.2 MB/s 
     |████████████████████████████████| 895 kB 17.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.6 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

model_name = "mrm8488/t5-base-finetuned-sarcasm-twitter" # you can specify the model size here
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
def eval_conversation(text):

  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt', max_length = 512)
  #print(input_ids)

  output = model.generate(input_ids=input_ids, max_length=3)
  
  dec = [tokenizer.decode(ids) for ids in output]

  label = dec[0]

  return label

# For similarity with the training dataset we should replace users mentions in twits for @USER token and urls for URL token.


In [ ]:
twit1 = "Trump just suspended the visa program that allowed me to move to the US to start @USER!"+" Unfortunately, I won’t be able to vote in a few months but if you can, please vote him out, "+"he's destroying what made America great in so many different ways!"

twit2 = "@USER @USER @USER We have far more cases than any other country, "+"so leaving remote workers in would be disastrous. Makes Trump sense."

twit3 = "My worry is that i wouldn’t be surprised if half the country actually agrees with this move..."

me = "Trump doing so??? It must be a mistake... XDDD"

conversation = twit1 + twit2
z = eval_conversation(conversation)
print(z) #Output: 'derison'

conversation = twit1 + twit3

#print(eval_conversation(conversation)) #Output: 'normal'

conversation = twit1 + me

print(eval_conversation(conversation)) #Output: 'derison'
# We will get 'normal' when sarcasm is not detected and 'derison' when detected


<pad> derison
<pad> derison


In [ ]:
fin = eval_conversation(conversation)
print(type(fin))
print(fin)

<class 'str'>
<pad> derison


In [ ]:
path_to_json_file = 'Reddit_Twitter_Dataset/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl'

In [ ]:
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [ ]:
type(json_data)

list

In [ ]:
for i in range(5):
  print(json_data[i])
  print(type(json_data[i]))
  print(type(json_data[i]['context']))

{'label': 'NOT_SARCASM', 'context': ['Well now that ’ s problematic AF <URL>', '@USER @USER My 5 year old ... asked me why they are making fun of Native Americans ..', "@USER @USER @USER I will take shit that didn't happen for $ 100", '@USER @USER @USER No .. he actually in the gifted program and reads on second grade level .  ... and he knows Kansas City is in Missouri'], 'response': '@USER @USER @USER My 3 year old , that just finished reading Nietzsche and then asked me : " ayo papa why these people always trying to cancel someone on Twitter , trying to pretend like that makes them better themselves ? " . To which I replied " idk " , and he just " cuz hoes mad " . Im so proud . <URL>', 'id': 'twitter_1'}
<class 'dict'>
<class 'list'>
{'label': 'SARCASM', 'context': ['Last week the Fake News said that a section of our powerful , under construction , Southern Border Wall “ fell over ” , trying to make it sound terrible , except the reason was that the concrete foundation was just pour

In [ ]:
from tqdm import tqdm
accurate = 0
tot_tweets = 0

# i=0
for tweet in tqdm(json_data):
  i+=1
  tru_label = 0
  if tweet['label'] == 'SARCASM':
    tru_label = 1
  tot_cont = ""
  for con in tweet['context']:
    tot_cont+=(con + " ")
  tot_cont+=tweet['response']
  output = eval_conversation(tot_cont)
  sarc_flag=0
  if 'derison' in output:
    sarc_flag=1
  if sarc_flag == tru_label:
    accurate+=1
  tot_tweets+=1
  # print(tweet['label'])
  # print(output)
  # if i>=10:
  #   break

100%|██████████| 1800/1800 [21:38<00:00,  1.39it/s]


In [ ]:
print("Final accuracy on test dataset: ", accurate/tot_tweets)
print("Total number of accurate examples: ", accurate, "Total Tweets: ", tot_tweets)

Final accuracy on test dataset:  0.7238888888888889
Total number of accurate examples:  1303 Total Tweets:  1800
